<font size="3">
Celem projektu jest zaimplementowanie i przeprowadzanie analizy porównawczej trzech algorytmów wyznaczania wartości i wektorów własnych macierzy z pokazaniem co najmniej dwóch przykładów zastosowania w analizie danych (np. wyznaczanie minimum formy kwadratowej x^TAx, PCA, SVD, pinv, itp...)

Proponowane aspekty podlegające porównaniu:
<ul>
    <li>ogólność i stabilność metod (analiza dziedziny z możliwością zastosowania),</li>
    <li>zajętość pamięciowa,</li>
    <li>obciążenie procesora,</li>
    <li>wrażliwość na błędy danych,</li>
    <li>ograniczenia co do rozmiaru macierzy.</li>
</ul>

Proponowane algorytmy (proszę samodzielnie wybrać 3):
<ul>
    <li>QR,</li>
    <li>Metoda potęgowa,</li>
    <li>Householder tranformation,</li>
    <li>Lanczos algorithm,</li>
    <li>Jacobie eigenvalue algorithm,</li>
    <li>Locally Optimal Block Preconditioned Conjugate Gradient (LOBPCG)</li>
</ul>
Implementacja algorytmów musi być wykonana w sposób możliwie efektywny i kontrolowany.
Oznacza to, że należy dołożyć starań, aby uruchamiany kod był możliwie zoptymalizowany.

Raport końcowy powinien składać się z czterech części:

<ol>
    <li>wstępu precyzującego jakie algorytmy i w jakim języku zostały zaimplementowane;</li>
    <li>opisu badania, przedstawiającego wybrane przypadki testowe (rodzaje macierzy i jakie cechy tymi macierzami zamierzaliście testować), aspekty porównania oraz (najważniejsze) sposoby i warunki ich pomiaru; w warunkach pomiaru proszę uwzględnić parametry środowiska uruchomieniowego: rozmiar i prędkość pamięci, model procesora, model i typ dysku; niezbędne jest wyszczególnienie wszystkich zastosowanych optymalizacji algorytmów;</li>
    <li>sekcji z wynikami przedstawionymi w odpowiedniej formie wizualnej;</li>
    <li>podsumowania, odnoszącego się do wyników, obiektywnie zestawiających cechy charakterystyczne tych algorytmów; należy skonfrontować uzyskane wyniki ze spodziewanymi; warto odnieść się w dyskusji do aspektu związane z implementacją algorytmów.
    </li>
</ol>
W raporcie końcowym należy między innymi załączyć wykresy prezentujące przebiegi procesu zbiegania się do rozwiązania.

In [1]:
A = [1 1 4;
    2 56 1;
    5 9 12]

3×3 Array{Int64,2}:
 1   1   4
 2  56   1
 5   9  12

## Metoda Potęgowa

In [2]:
# using Pkg
# Pkg.add("BenchmarkTools")

In [3]:
using LinearAlgebra
using BenchmarkTools

In [4]:
function potegowa(A, l::Integer)
    n = size(A, 1)
    x = ones(n, 1)
    local x1
    for i in 1:l
        x1 = x  # why do we need x from step i-1 
        x = A * x
        x = x / norm(x)
    end
    λ = x'*A*x1/(x'*x1)  # why is x1 used here
end

potegowa (generic function with 1 method)

In [5]:
# @btime potegowa(A, 1000)

In [6]:
function potegowav2(A, l::Integer)
    n = size(A, 1)
    x = ones(Float64, n, 1)
    x1 = Array{Float64}
    for i in 1:l
        x1 = x  # why do we need x from step i-1 
        x = A * x
        x = x / norm(x)
    end
    λ = x'*A*x/(x'*x)
#     λ = x'*A*x1/(x'*x1)  # why is x1 used here
    return λ, x
end

potegowav2 (generic function with 1 method)

In [7]:
# potegowav2(A, 1000)

In [8]:
# @btime potegowav2(A, 100)

In [9]:
eigen(A)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
values:
3-element Array{Float64,1}:
 -0.5410990010433583
 13.26793353484656
 56.273165466196815
vectors:
3×3 Array{Float64,2}:
 -0.935035    0.307084   -0.0323698
  0.0268217  -0.0366269  -0.978724
  0.35354     0.950977   -0.202614

## Metoda Jacobiego

In [10]:
display(A)

3×3 Array{Int64,2}:
 1   1   4
 2  56   1
 5   9  12

In [11]:
function eye(n)
    return Matrix{Float64}(I,n,n)
end

function maxst(A)
    s = 1;
    t = 2;
    n = size(A,1);

    for c = 2:n
        for r = 1:c-1
            if abs(A[r,c]) > abs(A[s,t])
            s = r;
            t = c;
            end
        end
    end
    return s,t
end


maxst (generic function with 1 method)

In [12]:
function jacobi(A, l::Integer)
    n = size(A,1);
    for i = 1:l
        s,t = maxst(A);
        d = sqrt((A[s,s] - A[t,t])^2 + 4*A[s,t]^2);
        sin2t = 2*A[s,t]/d;
        cos2t = (A[s,s] - A[t,t]) / d;
        dt = sqrt(2*(1+cos2t));
        sint = abs(sin2t) / dt;
        cost = abs((1+ cos2t) / dt);
        cost = sign(A[s,t]) * cost;

        R = eye(n);
        R[s,s] = cost;
        R[t,t] = cost;
        R[s,t] = -sint;
        R[t,s] = sint;
        A = R'*A*R;
    end
    return A
end

jacobi (generic function with 1 method)

In [13]:
AJ = jacobi(A, 15)
display(AJ)
diag(AJ)[end:-1:1]

3×3 Array{Float64,2}:
 56.2732    0.0        3.08084e-38
  7.15356  13.2679    -1.92593e-34
  3.79269   0.664874  -0.541099

3-element Array{Float64,1}:
 -0.5410990010433568
 13.267933534846554
 56.273165466196815

In [14]:
eigen(A)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
values:
3-element Array{Float64,1}:
 -0.5410990010433583
 13.26793353484656
 56.273165466196815
vectors:
3×3 Array{Float64,2}:
 -0.935035    0.307084   -0.0323698
  0.0268217  -0.0366269  -0.978724
  0.35354     0.950977   -0.202614

## Metoda QR

In [15]:
function mgs(A)
    n = size(A,1);
    R = zeros(n, n);
    Q = zeros(n, n);
    for j = 1:n
        v = A[:,j];
        for i = 1:j-1
            R[i,j] = Q[:,i]'*v;
            v = v - R[i,j]*Q[:,i];
        end
        R[j,j] = norm(v)
        Q[:,j] = v / R[j,j]
    end
    return Q,R
end



mgs (generic function with 1 method)

In [16]:
# AS = [1 5 6;
#     5 23 9;
#     6 9 18]

In [17]:
test, test2 = mgs(A);

In [18]:
display(test)
display(test2)

3×3 Array{Float64,2}:
 0.182574  -0.0873505   0.979304
 0.365148   0.930829    0.0149512
 0.912871  -0.354862   -0.201841

3×3 Array{Float64,2}:
 5.47723  28.8467  12.0499
 0.0      48.8453  -3.67691
 0.0       0.0      1.51007

In [19]:
function QR_eigen(A, l::Integer)
    Q,R = mgs(A)
    for k = 1:l
        Q,R = mgs(A);
        A = R*Q;
    end
    A
end

QR_eigen (generic function with 1 method)

In [20]:
# function QR_eigen_rand(l::Integer)
#     A = rand(5, 5);
#     A = A - tril(A,-1)
#     display(eigen(A))
#     Q,R = mgs(A)
#     for k = 1:l
#         Q,R = mgs(A);
#         A = R*Q;
#     end
#     diag(A)[end:-1:1]
#     A
# end

In [21]:
# QR_eigen_rand(100)

In [22]:
QR_eigen(A, 15)

3×3 Array{Float64,2}:
 56.2732        7.27402      3.58067
  1.45273e-7   13.2679      -0.517151
  2.00205e-28  -5.3929e-20  -0.541099

In [23]:
diag(QR_eigen(A, 15))[end:-1:1]

3-element Array{Float64,1}:
 -0.5410990010433575
 13.267933559418443
 56.273165441624926

In [24]:
diag(QR_eigen(A, 150))[end:-1:1]

3-element Array{Float64,1}:
 -0.5410990010433575
 13.267933534846561
 56.27316546619681

In [25]:
eigen(A)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
values:
3-element Array{Float64,1}:
 -0.5410990010433583
 13.26793353484656
 56.273165466196815
vectors:
3×3 Array{Float64,2}:
 -0.935035    0.307084   -0.0323698
  0.0268217  -0.0366269  -0.978724
  0.35354     0.950977   -0.202614